In [1]:
import re

import pandas as pd
import nltk
from nltk.tokenize import word_tokenize,wordpunct_tokenize,sent_tokenize
import spacy 
import es_core_news_sm

## Exploración

Los datos están en un libro de excel con múltiples hojas. Cada hoja corresponde a cada unidad de la universidad y otra hoja con el reporte general.
Cada una de las hojas se transformaran a formato csv.

In [2]:
excel = pd.read_excel("noticias/2025-05-12/2025-05-12.xlsx", sheet_name=None)
print(f"Nombre de las hojas del libro de excel:\n {[hoja for hoja in excel.keys()]}")

Nombre de las hojas del libro de excel:
 ['Reporte General', 'Azcapotzalco', 'Cuajimalpa', 'Iztapalapa', 'Lerma', 'Xochimilco']


In [3]:
for hoja, df in excel.items():
    nombre = hoja.lower().strip().replace(" ", "")
    output_path = f"noticias/2025-05-12/{nombre}.csv"
    df.to_csv(output_path, index=False)
    print(f"{output_path} creado.")
print("Todas las hojas creadas")

noticias/2025-05-12/reportegeneral.csv creado.
noticias/2025-05-12/azcapotzalco.csv creado.
noticias/2025-05-12/cuajimalpa.csv creado.
noticias/2025-05-12/iztapalapa.csv creado.
noticias/2025-05-12/lerma.csv creado.
noticias/2025-05-12/xochimilco.csv creado.
Todas las hojas creadas


En realidad la hoja que nos importa es la de Reporte General; en ella se agregan la info de todas las demás unidades y además tiene una columna nueva llamada Alcance, que nos ayudará a deifinir los rangos para clasificar las noticias.

In [4]:
reporte = pd.read_csv("noticias/2025-05-12/reportegeneral.csv")
reporte.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1269 entries, 0 to 1268
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Fecha    1269 non-null   object
 1   Medio    1269 non-null   object
 2   Título   1269 non-null   object
 3   Resumen  1262 non-null   object
 4   Alcance  1035 non-null   object
dtypes: object(5)
memory usage: 49.7+ KB


In [5]:
reporte.sample(5)

,Fecha,Medio,Título,Resumen,Alcance
288,2025-06-05,88.9 Noticias (Sitio),Alertan por los males que ocasionan alimentos ...,"Rafael Díaz García, del Departamento de Atenci...",162730
841,2025-05-24,La Jornada BC (sitio),Realizan ayuno de 24 horas en protesta por gen...,Por: Arturo Sánchez Jiménez,97400
954,2025-05-21,JLA Noticias CDMX (Sitio),UAM abre convocatoria para pase directo a 57 o...,La Universidad Autónoma Metropolitana lanzó la...,18200
1099,2025-05-20,Canal 21 Capital 21,Asesinan a dos colaboradores de la jefa de Gob...,En un ataque directo ocurrido esta mañana en l...,25000
1048,2025-05-21,La Hoguera (Sitio),"¿Quiénes eran Ximena Guzmán y José Muñoz, cola...","Este martes 20 de mayo, Ciudad de México (CDMX...",21400



Se puede ver que hay valores vacíos en la columna Alcance, y valores de error #REF!. Se procede a eliminarlos.

In [6]:
noerrors = reporte.copy()
noerrors.Alcance = pd.to_numeric(reporte.Alcance, errors="coerce") # this will convert to float cause int can't represent NaN
noerrors = noerrors.dropna(subset=["Alcance"])
noerrors.Alcance = noerrors.Alcance.astype(int)

# de una vez convertimos la columna Fecha a datetime
noerrors.Fecha = pd.to_datetime(noerrors.Fecha)
noerrors.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1033 entries, 1 to 1268
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   Fecha    1033 non-null   datetime64[ns]
 1   Medio    1033 non-null   object        
 2   Título   1033 non-null   object        
 3   Resumen  1026 non-null   object        
 4   Alcance  1033 non-null   int64         
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 48.4+ KB


In [7]:
noerrors.sample(10)

,Fecha,Medio,Título,Resumen,Alcance
787,2025-05-26,Demócrata Coahuila (sitio),"Presentan en la FCQ de la UAdeC el libro ""Bioc...","Saltillo, Coah., 26 de mayo 2025.- La Universi...",4000
634,2025-05-29,Infobae América (Sitio),Línea 8 Metro CDMX: por qué desalojaron a usua...,"Este jueves 29 de mayo, pasajeros fueron desal...",1120000
957,2025-05-21,Milenio (sitio),Difunden foto del presunto asesino de Ximena G...,El periodista y conductor del programa C4 en A...,199654
387,2025-06-03,88.1 Universal,"La calidad del agua en la Ciudad de México, un...",La calidad del agua potable en la Ciudad de Mé...,1404000
251,2025-06-06,Ciudad VLC VE (Sitio),"""Nos vamos contando historias"" por Mirih Berbin",Cuántas veces hemos iniciado una travesía pens...,116928
938,2025-05-22,Columna Digital CDMX (sitio),"¿De la UAM a la Justicia? Mauricio Tortolero, ...",El próximo nombre que se sumará a la boleta mo...,17800
198,2025-06-07,Billie Parker Noticias (sitio),"Martín Aguilar, el rector de la UV, que quiere...",*Místicos y Terrenales,22700
1092,2025-05-20,Canal 6 Multimedios,Ximena Guzmán Cuevas y José Muñoz Vela fueron ...,"Ximena Guzmán Cuevas, secretaria particular de...",442055
415,2025-06-03,Sedema CDMX (sitio),"Inscríbete y participa en el ""Concurso Gastron...","La Secretaría del Medio Ambiente (SEDEMA), en ...",482000
447,2025-06-02,Reforma.com (sitio),"Jardín Xochitlalyocan, un paraíso medicinal","Junto al Antiguo Canal de Cuemanco, en pleno p...",8200000


¿Cuántos datos tuvimos que eliminar en el proceso de limpiado?

In [8]:
print(f"Aún nos quedamos con el {(len(noerrors) / len(reporte)):.0%} de la data original.\nCorresponde a {len(noerrors)} registros.")

Aún nos quedamos con el 81% de la data original.
Corresponde a 1033 registros.


In [9]:
noerrors.to_csv("./noticias/2025-05-12/noerrors.csv")

In [10]:
print(f"El alcance de las noticias está en el rango de {noerrors.Alcance.min()} a {noerrors.Alcance.max():,}")

step = int((noerrors.Alcance.max() - noerrors.Alcance.min()) / 3) + 1
impact_levels = ["low_impact", "mid_impact", "big_impact"]
levels = {level: [noerrors.Alcance.min() + i*step, noerrors.Alcance.min() + (i+1)*step] for i, level in enumerate(impact_levels)}
levels

El alcance de las noticias está en el rango de 13 a 23,300,000


{'low_impact': [np.int64(13), np.int64(7766676)],
 'mid_impact': [np.int64(7766676), np.int64(15533339)],
 'big_impact': [np.int64(15533339), np.int64(23300002)]}

In [11]:
print("Resumen de la noticia con más alcance:")
for r in noerrors.loc[noerrors.Alcance == noerrors.Alcance.max()].Resumen:
    print(r)
    
print("\nResumen de la noticia con menos alcance:")
for r in noerrors.loc[noerrors.Alcance == noerrors.Alcance.min()].Resumen:
    print(r)

Resumen de la noticia con más alcance:
"Socióloga del deporte, corredora de ida y de regreso, experimentando en la bici, viajera y amante del precipicio", así era como se describía así misma Ximena Guzmán Cuevas, secretaria particular de Clara Brugada, quien lamentablemente perdió la vida esta mañana durante un ataque armado directo por sicarios abordo de una motocicleta en Tlalpan, Ciudad de México.

Resumen de la noticia con menos alcance:
Foto: Dirección General de Comunicación Social UNAM


In [12]:
noerrors.loc[noerrors.Alcance == noerrors.Alcance.min()]

,Fecha,Medio,Título,Resumen,Alcance
782,2025-05-26,Página 13 (Sitio),Universidades mexicanas firman Declaratoria Na...,Foto: Dirección General de Comunicación Social...,13


## Consideraciones y Opciones a futuro
- <mark>Solo se tomará en cuenta la columna Resumen</mark> para clasificar por alcance. Hay algunas noticias con resumen mucho muy reducido; tendrá que ser necesario visitar el hipervínculo para acceder a todo el artículo.

## Limpieza
- Eliminar menciones de usuarios (@user)
- Eliminar caracteres especiales +/?!¡;
- Eliminar correos electrónicos
- Eliminar direcciones URL

In [16]:
resumenes = noerrors.Resumen.copy()
resumenes

1       Este jueves 12 de junio se tienen contempladas...
2       *Participaron en esta edición más de 350 perso...
3       Académicos de la Universidad Autónoma Metropol...
4       La contaminación plástica se ha convertido en ...
5       ¡Toma precauciones! Checa dónde y a qué hora h...
                              ...                        
1263    La Secretaría de Cultura del Gobierno de Méxic...
1264    *Parte de su legado pictórico hoy se conserva ...
1266    La diputada Cynthia Delgado Mendoza informó qu...
1267    "Cuando a la UNAM, a la UAM y la Universidad A...
1268    CIUDAD VICTORIA, TAMPS. Con una actuación impe...
Name: Resumen, Length: 1033, dtype: object

In [17]:
usuarios = r"(?<!\w)@\w+"
email = r"\w+@\w+[.\w]+(?=\s)"
especiales = r"[+/?!¡;\(\)\.\-]"
url = r"https?://.+(?=\s)"
todos = f"({url})|({email})|({usuarios})|({especiales})"

expr = re.compile(todos, re.M | re.I)
resumenes = resumenes.str.replace(
    pat=expr,
    repl="",
    regex=True
)

In [19]:
print(resumenes.sample(3))

1157    Con el firme respaldo del rector Dámaso Anaya ...
939     Contar con estudios universitarios no solo inc...
290                        Tlaxcala, a 29 de mayo de 2025
Name: Resumen, dtype: object


Note la ausencia de paréntesis entre el primer texto y el segundo.

## Normalizar
- Pasar todo a minúsculas

In [ ]:

textos_limpios = textos_limpios.str.lower()
textos_limpios

## Palabras vacías (Stopwords)

In [ ]:
from nltk.corpus import stopwords

palabras_vacias = set(stopwords.words("spanish"))

In [ ]:
def quitar_palabras_vacias(texto: str):
    tokens = texto.split()
    filtrado = [token for token in tokens if token not in palabras_vacias]
    return " ".join(filtrado)

In [ ]:
textos_limpios = textos_limpios.apply(quitar_palabras_vacias)
textos_limpios

In [ ]:
print(textos[3])
print("*" * 100)
print(textos_limpios[3])

Note la ausencia de artículos y de preposiciones.

## Segmentar 
- Unigramas

In [ ]:
unigramas = textos_limpios.apply(lambda texto: word_tokenize(texto, language="spanish")) 
unigramas[3]

In [ ]:
procesado = pd.DataFrame({
    "texto_original": textos,
    "unigramas": unigramas
})
procesado

## Lematización

In [ ]:
nlp = es_core_news_sm.load()

def crear_lemas(texto: str):
    return [palabra.lemma_ for palabra in nlp(texto)]

lemas = textos_limpios.apply(crear_lemas)
lemas[3]

## Derivación (Stemming)

In [ ]:
from nltk import SnowballStemmer

esp = SnowballStemmer("spanish")

def crear_stemming(tokens: list):
    tokens = list(tokens)
    return [esp.stem(token) for token in tokens]    

stemming = unigramas.apply(crear_stemming)
stemming[3]

## Resultados

In [ ]:
procesado = pd.DataFrame({
    "texto_original": textos,
    "texto_limpio": textos_limpios,
    "unigramas": unigramas,
    "lemas": lemas,
    "stemming": stemming,
})
procesado